In [1]:
train_dir = 'C:/Users/ROG/OneDrive/桌面/FYP/Dataset/Train_data/train_data_after_washing.csv'
test_dir = 'C:/Users/ROG/OneDrive/桌面/FYP/Dataset/Test_data/test_data_after_washing.csv'

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.models import Model
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score, cohen_kappa_score, roc_curve, auc, make_scorer, accuracy_score, f1_score 
tf.get_logger().setLevel('ERROR') # return ERROR messages, ignore others

In [3]:
#one hot
def encode_one_hot(ori_dataframe):
    dummies = pd.get_dummies(ori_dataframe)
    res = pd.concat([ori_dataframe, dummies], axis=1)
    return res

In [4]:
#load data
data = pd.read_csv(train_dir)
data = pd.DataFrame(data)
test_data = pd.read_csv(test_dir)
test_data = pd.DataFrame(test_data)
train_data = pd.DataFrame()
val_data = pd.DataFrame()
data = data[["review","rating"]]
test_data = test_data[["review","rating"]]

In [5]:
data['labels'] = data['rating'].map({1 : 0,
                                     2 : 0,
                                     3 : 0,
                                     4 : 0,
                                     5 : 1,
                                     6 : 1,
                                     7 : 1,
                                     8 : 1,
                                     9 : 2,
                                     10 : 2})

In [6]:
test_data['labels'] = test_data['rating'].map({1 : 0,
                                               2 : 0,
                                               3 : 0,
                                               4 : 0,
                                               5 : 1,
                                               6 : 1,
                                               7 : 1,
                                               8 : 1,
                                               9 : 2,
                                               10 : 2})

In [7]:
#extract data
data = data[["review","labels"]]
y = data["labels"].to_numpy()
Y = data["labels"]
#data = encode_one_hot(data)
test_data = test_data[["review","labels"]]

In [8]:
X = data["review"].to_numpy()
#Y = data[[0,1,2]].to_numpy
test_X = test_data["review"].to_numpy()
test_y = test_data["labels"].to_numpy()

In [10]:
def build_classifier_model(tfhub_handle_encoder,tfhub_handle_preprocess):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [12]:
import time
def model_training(x_train, y_train, x_val, y_val, times):
    times = str(times)
    tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/albert_en_base/2'
    tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/albert_en_preprocess/3'
    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)
    classifier_model = build_classifier_model(tfhub_handle_encoder, tfhub_handle_preprocess)
    classifier_model.summary()
    epochs = 150
    steps_per_epoch = 0
    for _ in x_train:
        steps_per_epoch = steps_per_epoch + 1
    num_train_steps = steps_per_epoch/24 * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-6
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    classifier_model.compile(optimizer=optimizer,
                             loss=loss,
                             metrics=["accuracy", tf.keras.metrics.Recall(name='recall')])
    file_name='TWB_Cross' + times + '_2_121_1.25'
    checkpoint_path = 'C:/Users/ROG/OneDrive/桌面/FYP/Model/'+ file_name + '/ckpt/cp.ckpt'
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    csv_callback = tf.keras.callbacks.CSVLogger(
    'C:/Users/ROG/OneDrive/桌面/FYP/Model/'+ file_name + '/record.csv', separator=',', append=False
    )
    t0 = time.time()
    print(f'Training model with {tfhub_handle_encoder}')
    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(x=x_train,
                                   y=y_train,
                                   validation_data=(x_val,y_val),
                                   epochs=150,
                                   batch_size=24,
                                   callbacks=[cp_callback,csv_callback])
    t1 = time.time()
    time_train = t1-t0
    return (classifier_model,time_train)

In [13]:
def graded_precision(y_true, y_pred, weights):
    precision_0 = precision_score(y_true, y_pred, labels=[0], average='macro')
    precision_1 = precision_score(y_true, y_pred, labels=[1], average='macro')
    precision_2 = precision_score(y_true, y_pred, labels=[2], average='macro')
    gp = ( weights[0] * precision_0 + weights[1] * precision_1 + weights[2] * precision_2 ) / ( weights[0] + weights[1] + weights[2] )
    return gp
def graded_recall(y_true, y_pred, weights):
    recall_0 = recall_score(y_true, y_pred, labels=[0], average='macro')
    recall_1 = recall_score(y_true, y_pred, labels=[1], average='macro')
    recall_2 = recall_score(y_true, y_pred, labels=[2], average='macro')
    gr = ( weights[0] * recall_0 + weights[1] * recall_1 + weights[2] * recall_2 ) / ( weights[0] + weights[1] + weights[2] )
    return gr
def graded_f1(precision, recall):
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [14]:
def model_testing(model, X, y, times,val):
    times = str(times)
    #generate y_true and prediction results
    y_true = y
    if val == True:
        y_true = np.argmax(y, axis=1)
    pred = np.argmax(model.predict(X),axis=1)
    
    #different metrics
    acc = accuracy_score(y_true, pred)
    weights = [2, 1, 1]
    prec = graded_precision(y_true, pred, weights)
    rec = graded_recall(y_true, pred, weights)
    f1 = graded_f1(prec, rec)
    kappa = cohen_kappa_score(y_true, pred)
    
    #CM
    if val == False:
        con_mat = confusion_matrix(y, pred)
        con_mat_norm = con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis]     # 归一化
        con_mat_norm = np.around(con_mat_norm, decimals=2)
        plt.figure(figsize=(8, 8))
        sns.heatmap(con_mat_norm, annot=True, cmap='Blues')
        plt.ylim(0, 3)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        #save CM
        file_name='TWB_Cross' + times + '_2_121_1.25'
        plt.savefig(fname='C:/Users/ROG/OneDrive/桌面/FYP/Model/'+ file_name + '/CM.png', dpi=300)
        plt.close()
    
    return (acc, prec, rec, f1, kappa)

In [15]:
#10-fold
skf = StratifiedKFold(n_splits=10)
val_acc = []
val_gp = []
val_gr = []
val_f1 = []
val_kp = []
tes_acc = []
tes_gp = []
tes_gr = []
tes_f1 = []
tes_kp = []
train_time = []
times = 0
for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = Y[train_index], Y[val_index]
    y_train = encode_one_hot(y_train)
    y_val = encode_one_hot(y_val)
    y_train = y_train[[0,1,2]].to_numpy()
    y_val = y_val[[0,1,2]].to_numpy()
    model,time_train = model_training(X_train, y_train, X_val, y_val, times)
    train_time.append(time_train)
    val = True
    acc, prec, rec, f1, kappa = model_testing(model, X_val, y_val, times, val)
    val_acc.append(acc)
    val_gp.append(prec)
    val_gr.append(rec)
    val_f1.append(f1)
    val_kp.append(kappa)
    val = False
    acc, prec, rec, f1, kappa = model_testing(model, test_X, test_y, times, val)
    tes_acc.append(acc)
    tes_gp.append(prec)
    tes_gr.append(rec)
    tes_f1.append(f1)
    tes_kp.append(kappa)
    times = times + 1

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

Epoch 12/150
1179/1179 [==============================] - ETA: 0s - loss: 0.2534 - accuracy: 0.9107 - recall: 0.9074
Epoch 12: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 300s 255ms/step - loss: 0.2534 - accuracy: 0.9107 - recall: 0.9074 - val_loss: 0.6540 - val_accuracy: 0.7718 - val_recall: 0.7670
Epoch 13/150
1179/1179 [==============================] - ETA: 0s - loss: 0.2148 - accuracy: 0.9278 - recall: 0.9254
Epoch 13: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 301s 255ms/step - loss: 0.2148 - accuracy: 0.9278 - recall: 0.9254 - val_loss: 0.6664 - val_accuracy: 0.7778 - val_recall: 0.7734
Epoch 14/150
1179/1179 [==============================] - ETA: 0s - loss: 0.1818 - accuracy: 0.9390 - recall: 0.9372
Epoch 14: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [

1179/1179 [==============================] - ETA: 0s - loss: 0.0046 - accuracy: 0.9988 - recall: 0.9988
Epoch 54: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 301s 255ms/step - loss: 0.0046 - accuracy: 0.9988 - recall: 0.9988 - val_loss: 1.8128 - val_accuracy: 0.8402 - val_recall: 0.8402
Epoch 55/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0054 - accuracy: 0.9989 - recall: 0.9989
Epoch 55: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 301s 255ms/step - loss: 0.0054 - accuracy: 0.9989 - recall: 0.9989 - val_loss: 1.7555 - val_accuracy: 0.8453 - val_recall: 0.8450
Epoch 56/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0059 - accuracy: 0.9988 - recall: 0.9988
Epoch 56: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [=============

1179/1179 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 0.9995 - recall: 0.9995
Epoch 96: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 300s 255ms/step - loss: 0.0021 - accuracy: 0.9995 - recall: 0.9995 - val_loss: 1.8726 - val_accuracy: 0.8447 - val_recall: 0.8447
Epoch 97/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 0.9996 - recall: 0.9996
Epoch 97: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 301s 255ms/step - loss: 0.0014 - accuracy: 0.9996 - recall: 0.9996 - val_loss: 1.8963 - val_accuracy: 0.8479 - val_recall: 0.8479
Epoch 98/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 0.9996 - recall: 0.9996
Epoch 98: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [=============

Epoch 117/150
1179/1179 [==============================] - ETA: 0s - loss: 8.0818e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 117: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 302s 256ms/step - loss: 8.0818e-04 - accuracy: 0.9997 - recall: 0.9997 - val_loss: 1.9247 - val_accuracy: 0.8466 - val_recall: 0.8463
Epoch 118/150
1179/1179 [==============================] - ETA: 0s - loss: 8.2024e-04 - accuracy: 0.9996 - recall: 0.9996
Epoch 118: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 302s 256ms/step - loss: 8.2024e-04 - accuracy: 0.9996 - recall: 0.9996 - val_loss: 1.9047 - val_accuracy: 0.8450 - val_recall: 0.8450
Epoch 119/150
1179/1179 [==============================] - ETA: 0s - loss: 7.3540e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 119: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross0_2_121_1.2

...

Epoch 20/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0745 - accuracy: 0.9841 - recall: 0.9840
Epoch 20: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 0.0745 - accuracy: 0.9841 - recall: 0.9840 - val_loss: 1.2149 - val_accuracy: 0.8376 - val_recall: 0.8373
Epoch 21/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0611 - accuracy: 0.9881 - recall: 0.9881
Epoch 21: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 0.0611 - accuracy: 0.9881 - recall: 0.9881 - val_loss: 1.2433 - val_accuracy: 0.8344 - val_recall: 0.8341
Epoch 22/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0614 - accuracy: 0.9883 - recall: 0.9883
Epoch 22: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [

1179/1179 [==============================] - ETA: 0s - loss: 0.0110 - accuracy: 0.9980 - recall: 0.9980
Epoch 62: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 302s 257ms/step - loss: 0.0110 - accuracy: 0.9980 - recall: 0.9980 - val_loss: 1.8517 - val_accuracy: 0.8488 - val_recall: 0.8488
Epoch 63/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0104 - accuracy: 0.9980 - recall: 0.9980
Epoch 63: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 0.0104 - accuracy: 0.9980 - recall: 0.9980 - val_loss: 1.8593 - val_accuracy: 0.8507 - val_recall: 0.8507
Epoch 64/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0124 - accuracy: 0.9979 - recall: 0.9979
Epoch 64: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [=============

1179/1179 [==============================] - ETA: 0s - loss: 0.0025 - accuracy: 0.9995 - recall: 0.9995
Epoch 104: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 0.0025 - accuracy: 0.9995 - recall: 0.9995 - val_loss: 2.0020 - val_accuracy: 0.8529 - val_recall: 0.8529
Epoch 105/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0018 - accuracy: 0.9995 - recall: 0.9995
Epoch 105: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 302s 257ms/step - loss: 0.0018 - accuracy: 0.9995 - recall: 0.9995 - val_loss: 1.9926 - val_accuracy: 0.8558 - val_recall: 0.8558
Epoch 106/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0022 - accuracy: 0.9996 - recall: 0.9996
Epoch 106: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [========

Epoch 125/150
1179/1179 [==============================] - ETA: 0s - loss: 8.2620e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 125: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 8.2620e-04 - accuracy: 0.9997 - recall: 0.9997 - val_loss: 2.0388 - val_accuracy: 0.8558 - val_recall: 0.8558
Epoch 126/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0010 - accuracy: 0.9997 - recall: 0.9997
Epoch 126: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 0.0010 - accuracy: 0.9997 - recall: 0.9997 - val_loss: 2.0349 - val_accuracy: 0.8555 - val_recall: 0.8555
Epoch 127/150
1179/1179 [==============================] - ETA: 0s - loss: 0.0010 - accuracy: 0.9997 - recall: 0.9997
Epoch 127: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ck

Epoch 146/150
1179/1179 [==============================] - ETA: 0s - loss: 7.4317e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 146: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 7.4317e-04 - accuracy: 0.9997 - recall: 0.9997 - val_loss: 2.0384 - val_accuracy: 0.8529 - val_recall: 0.8529
Epoch 147/150
1179/1179 [==============================] - ETA: 0s - loss: 8.1905e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 147: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.25/ckpt\cp.ckpt
1179/1179 [==============================] - 303s 257ms/step - loss: 8.1905e-04 - accuracy: 0.9997 - recall: 0.9997 - val_loss: 2.0403 - val_accuracy: 0.8532 - val_recall: 0.8532
Epoch 148/150
1179/1179 [==============================] - ETA: 0s - loss: 8.3879e-04 - accuracy: 0.9997 - recall: 0.9997
Epoch 148: saving model to C:/Users/ROG/OneDrive/桌面/FYP/Model/TWB_Cross9_2_121_1.2

In [16]:
matrics_list = [val_acc, val_gp, val_gr, val_f1, val_kp, tes_acc, tes_gp, tes_gr, tes_f1, tes_kp, train_time]
avg_results = []
for matric in matrics_list:
    total = 0
    for item in matric:
        total = total + item
    avg_results.append(total/len(matric))

In [17]:
avg_results

[0.846971787072721,
 0.8557234314585903,
 0.8382299645828357,
 0.846861882763587,
 0.7508432514684216,
 0.8517922177496645,
 0.8590107103373805,
 0.8439132680942494,
 0.851375334178203,
 0.7594025930074606,
 45303.34133560657]

In [18]:
with open("C:/Users/ROG/OneDrive/桌面/FYP/Model/BERT_new_crossloss.txt", "w") as f:
    f.write("val_acc: ")
    for item in val_acc:
        f.write(str(item))
        if val_acc.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")
    
    f.write("val_gp: ")
    for item in val_gp:
        f.write(str(item))
        if val_gp.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")
    
    f.write("val_gr: ")
    for item in val_gr:
        f.write(str(item))
        if val_gr.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
        
    f.write("val_f1: ")
    for item in val_f1:
        f.write(str(item))
        if val_f1.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
    
    f.write("val_kp: ")
    for item in val_kp:
        f.write(str(item))
        if val_kp.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")   
    
    f.write("tes_acc: ")
    for item in tes_acc:
        f.write(str(item))
        if tes_acc.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
        
    f.write("tes_gp: ")
    for item in tes_gp:
        f.write(str(item))
        if tes_gp.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
        
    f.write("tes_gr: ")
    for item in tes_gr:
        f.write(str(item))
        if tes_gr.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
            
    f.write("tes_f1: ")
    for item in tes_f1:
        f.write(str(item))
        if tes_f1.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")  
            
    f.write("tes_kp: ")
    for item in tes_kp:
        f.write(str(item))
        if tes_kp.index(item) == len(val_acc) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")
    
    f.write("train_time: ")
    for item in train_time:
        f.write(str(item))
        if train_time.index(item) == len(train_time) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")
    
    f.write("average_results: ")
    for item in avg_results:
        f.write(str(item))
        if avg_results.index(item) == len(avg_results) - 1: # Check if last item
            f.write(';')
        else:
            f.write(', ')
    f.write("\n")

In [19]:
avg_results

[0.846971787072721,
 0.8557234314585903,
 0.8382299645828357,
 0.846861882763587,
 0.7508432514684216,
 0.8517922177496645,
 0.8590107103373805,
 0.8439132680942494,
 0.851375334178203,
 0.7594025930074606,
 45303.34133560657]